#### 변수 타입 확인 방법
- 사용 데이터: Telco_churn_prediction.csv

In [1]:
# 데이터 경로 설정 및 데이터 불러오기
import os
os.chdir('./data')

import pandas as pd
df = pd.read_csv("Telco_churn_prediction.csv")

In [2]:
df.dtypes # TotalCharges 타입이 object임 (도메인 지식과 상충)

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [3]:
df.infer_objects().dtypes 
# 추론 결과도 TotalCharges 타입이 object임

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
df['TotalCharges'] 
# 아무리봐도 연속형 변수 같음 => 문자가 섞여 있을 수 있겠다란 결론

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: TotalCharges, Length: 7043, dtype: object

In [5]:
def find_str_element(val):
    try:
        float(val) # 만약 val이 문자라면 여기서 오류가 발생할 것이므로 except로 넘어감
        return False
    except:
        return True

# 공백이 섞여 있음을 확인
# apply 함수는 자주 사용되는 굉장히 중요한 함수이므로 반드시 숙지
# apply 함수는 각 요소에 함수를 일괄 적용하는 함수임
print(df['TotalCharges'][df['TotalCharges'].apply(find_str_element)].values)

[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']


In [6]:
# apply 사용 예제 (for문보다 훨씬 빨라서 자주 사용됨)
def f(x):
    return x**2

import pandas as pd
S = pd.Series([1, 2, 3, 4]) 
S.apply(f)

0     1
1     4
2     9
3    16
dtype: int64

In [7]:
def getting_unique_val(col):
    return len(col.unique())

df.apply(getting_unique_val, axis = 0) 
# 유니크한 값 개수 기준으로 판단
# 일반적으로 그 개수가 많으면 연속형, 그렇지 않으면 범주형인 경우가 대다수 (ID 관련 컬럼 제외)

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

#### 혼합형 변수가 존재하는 경우

In [8]:
# 데이터 불러오기
import pandas as pd
df = pd.read_csv("baseball.csv")

In [9]:
# 특징과 라벨 분리
X = df.drop('Salary', axis = 1)
Y = df['Salary']

In [19]:
df.dtypes

Batting_average            float64
On-base_percentage         float64
Runs                         int64
Hits                         int64
Doubles                      int64
Triples                      int64
HomeRuns                     int64
Runs_batted_in               int64
Walks                        int64
Strike-Outs                  int64
Stolen_bases                 int64
Errors                       int64
Free_agency_eligibility      int64
Free_agent                   int64
Arbitration_eligibility      int64
Arbitration                  int64
Salary                       int64
dtype: object

In [10]:
# 모델 리스트 정의
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.linear_model import LinearRegression as LR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.neighbors import KNeighborsRegressor as KNR

# 공정한 비교를 위해 전부 default 값을 사용
# random_state가 있는 모델은 모두 같은 값으로 설정

MLP_model = MLP(random_state = 100)
LR_model = LR()
DTR_model = DTR(random_state = 100)
RFR_model = RFR(random_state = 100)
KNR_model = KNR()

model_list = [MLP_model, LR_model, DTR_model, RFR_model, KNR_model]
model_name_list = ['MLP', 'LR', 'DTR', 'RFR', 'KNR']

In [17]:
# 모델별 k겹 교차 검증 기반(k = 5)의 MAE값 계산
from sklearn.model_selection import cross_val_score
for (model, model_name) in zip(model_list, model_name_list):
    score = -cross_val_score(model, X, Y, cv = 5, scoring = 'neg_mean_absolute_error').mean() # -MAE이므로 다시 -를 붙인 것
    print(model_name, score)
    
# 아래 ConvergenceWarning: 신경망이 지역최솟값에 도달하지 못했을 때(정상적으로 학습X)
# => max_iter를 올려주면 경고를 피할 수 있으나 지금은 공정한 비교를 위해

C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packa

MLP 710.1436141313578
LR 539.567114538396
DTR 573.3383230904303
RFR 467.4962985074626
KNR 651.0923792800702


- 좋게 나온 모델: RFR
- 나쁘게 나온 모델: MLP(얘는 수렴하지 않았기때문에 성능이 안 좋은 것처럼 나온 것), KNN
    - 데이터: 연속형+이진형+정수형
    - MLP는 ConvergenceWarning을 피하면 성능이 제법 좋게 나올 것같음